In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#basics
import sys,os

#utils
import paths

#main libraries
import pandas as pd

#model Libraries
import sklearn.linear_model


In [3]:
#private modules
from src.data import make_dataset

from src.utils import memory_usage

In [4]:
memory_usage.memory()

Available memory (%): 31.290912628173828


In [5]:
%%time
df_training,features,target = make_dataset.get_data(nrows=5000, low_memory=False, dataset="training")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Columns: 314 entries, id to target_kazutsugi
dtypes: float64(311), object(3)
memory usage: 12.0+ MB
None
CPU times: user 769 ms, sys: 431 ms, total: 1.2 s
Wall time: 8.56 s


In [22]:
%%time
df_tournament,features,target = make_dataset.get_data(nrows=5000, low_memory=False, dataset="tournament")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Columns: 314 entries, id to target_kazutsugi
dtypes: float64(311), object(3)
memory usage: 12.0+ MB
None
CPU times: user 1.37 s, sys: 895 ms, total: 2.27 s
Wall time: 18.7 s


In [59]:
%%time
import sklearn.linear_model
model = sklearn.linear_model.LinearRegression()
model.fit(df_training[features], df_training[target])

CPU times: user 2.71 s, sys: 85.3 ms, total: 2.8 s
Wall time: 1.61 s


LinearRegression()

In [25]:
%%time
from sklearn import preprocessing
from sklearn import utils

from sklearn.ensemble import RandomForestClassifier

#encoding target to multiclass
lab_enc = preprocessing.LabelEncoder()
encoded_target = lab_enc.fit_transform(df_training[target])

model = RandomForestClassifier(n_estimators=10, n_jobs=-1)
model.fit(df_training[features], encoded_target)

CPU times: user 5.92 s, sys: 278 ms, total: 6.2 s
Wall time: 2.47 s


RandomForestClassifier(n_estimators=10, n_jobs=-1)

In [81]:
%%time
from joblib import parallel_backend

with parallel_backend('threading', n_jobs=-1):
    # Train the model
    model = RandomForestClassifier(n_estimators=1000, n_jobs=-1)
    model.fit(df_training[features], encoded_target)
    # Test the model
    #y_pred = clf.predict(X_test)


1.99 s ± 148 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%time
from dask.distributed import Client

client = Client()  # Connect to a Dask Cluster

with parallel_backend('dask', n_jobs=-1):
    # Train the model
    model = RandomForestClassifier(n_estimators=1000, n_jobs=-1)
    model.fit(df_training[features], encoded_target)
    # Test the model
    #y_pred = clf.predict(X_test)

In [6]:
%%time
from sklearn import preprocessing
from sklearn import utils

from sklearn.ensemble import RandomForestClassifier

#encoding target to multiclass
lab_enc = preprocessing.LabelEncoder()
encoded_target = lab_enc.fit_transform(df_training[target])

model = RandomForestClassifier(n_estimators=10, n_jobs=-1)
model.fit(df_training[features], encoded_target)

CPU times: user 5.66 s, sys: 207 ms, total: 5.86 s
Wall time: 2.14 s


RandomForestClassifier(n_estimators=10, n_jobs=-1)

In [23]:
%%time
import numpy as np
from dask.distributed import Client

import joblib
from sklearn.datasets import load_digits
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC

client = Client(processes=False)             # create local cluster


param_space =  {
    #'bootstrap' : [True, False],
    #'ccp_alpha' : 0.0, 
    #'class_weight': [None, {0:1,1:1}, {0:1,1:10}, {0:1,1:100}, 'balanced'],
    #'criterion' : 'gini', 
    #'max_depth' : [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None], 
    #'max_features' : ['auto', 'sqrt'],
    #'max_leaf_nodes' : None,
    #'max_samples' : None,
    #'min_impurity_decrease' : 0.0, 
    #'min_impurity_split' : None,
    #'min_samples_leaf' : [1, 2, 4],
    #'min_samples_split' : [2, 5, 10],
    #'min_weight_fraction_leaf' : 0.0, 
    'n_estimators' : [1000, 100],#200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000],
    'n_jobs' : [-1], 
    #'oob_score' : False, 
    #'random_state' : None,
    #'verbose' : 0, 
    #'warm_start' : False
}



model = RandomForestClassifier()
search = RandomizedSearchCV(model, param_space, cv=3, n_iter=2, verbose=1)

with joblib.parallel_backend('dask'):
    search.fit(df_training[features], encoded_target)

/Users/nicholasrichers/Documents/virtualenvs/numerai_env/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 54822 instead
  warnings.warn(
[Parallel(n_jobs=-1)]: Using backend DaskDistributedBackend with 4 concurrent workers.


Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  6.6min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  6.6min finished


CPU times: user 32min 56s, sys: 1min 7s, total: 34min 4s
Wall time: 9min 22s


In [24]:
%%time
import numpy as np
#from dask.distributed import Client

import joblib
from sklearn.model_selection import RandomizedSearchCV
#from sklearn.svm import SVC

#client = Client(processes=False)             # create local cluster



param_space =  {
    #'bootstrap' : [True, False],
    #'ccp_alpha' : 0.0, 
    #'class_weight': [None, {0:1,1:1}, {0:1,1:10}, {0:1,1:100}, 'balanced'],
    #'criterion' : 'gini', 
    #'max_depth' : [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None], 
    #'max_features' : ['auto', 'sqrt'],
    #'max_leaf_nodes' : None,
    #'max_samples' : None,
    #'min_impurity_decrease' : 0.0, 
    #'min_impurity_split' : None,
    #'min_samples_leaf' : [1, 2, 4],
    #'min_samples_split' : [2, 5, 10],
    #'min_weight_fraction_leaf' : 0.0, 
    'n_estimators' : [1000, 100],#200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000],
    'n_jobs' : [-1], 
    #'oob_score' : False, 
    #'random_state' : None,
    #'verbose' : 0, 
    #'warm_start' : False
}



model = RandomForestClassifier()
search = RandomizedSearchCV(model, param_space, cv=3, n_iter=2, verbose=1)

 #with joblib.parallel_backend('dask'):
search.fit(df_training[features], encoded_target)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  7.0min finished


CPU times: user 12min 13s, sys: 1min 10s, total: 13min 23s
Wall time: 9min 49s


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=2,
                   param_distributions={'n_estimators': [1000, 100],
                                        'n_jobs': [-1]},
                   verbose=1)

In [17]:
df_training,features,target = make_dataset.get_data(nrows=50000, low_memory=False, dataset="training")
lab_enc = preprocessing.LabelEncoder()
encoded_target = lab_enc.fit_transform(df_training[target])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Columns: 314 entries, id to target_kazutsugi
dtypes: float64(311), object(3)
memory usage: 119.8+ MB
None
